In [91]:
function importfrom(moduleinstance::Module, functionname::Symbol, argtypes::Tuple)
    meths = methods(moduleinstance.(functionname), argtypes)
    importfrom(moduleinstance, functionname, meths)
end 

function importfrom(moduleinstance::Module, functionname::Symbol)
    meths = methods(moduleinstance.(functionname))
    importfrom(moduleinstance, functionname, meths)
end 

function importfrom(moduleinstance::Module, functionname::Symbol, meths::Base.MethodList)
    for mt in meths
        paramnames = collect(mt.lambda_template.slotnames[2:end])
        paramtypes = collect(mt.sig.parameters[2:end])
        paramsig = ((n,t)->Expr(:(::),n,t)).(paramnames, paramtypes)

        funcdec = Expr(:(=), 
                        Expr(:call, functionname, paramsig...),
                        Expr(:call, :($moduleinstance.$functionname), paramnames...)
        )
        eval(funcdec) #Runs at global scope
        
    end
end

importfrom (generic function with 3 methods)

In [92]:
using ProgressMeter
using WordNet
using AdaGram
using AdaGramCompat
using SwiftObjectStores
using CorpusLoaders
using CorpusLoaders.Semcor
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using JLD
importfrom(CorpusLoaders.Semcor, :sensekey)
importfrom(WordNet, :sensekey)

In [3]:
all_word_sense_vectors = AdaGramCompat.all_word_sense_vectors

all_word_sense_vectors (generic function with 1 method)

In [4]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [5]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
#am = AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)
#am = open(deserialize,"models/adagram/more_senses.adagram_model.jsz", "r");
am = load("models/adagram/more_senses.adagram_model.jld", "am");


In [ ]:
s_am = get_jld(SwiftService(), "sensemodels/adagram/", "semhuff_more_senses.adagram_model.jld", "am");

In [ ]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [6]:
using CorpusLoaders.Semeval2007t7
challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");
semcor = index_semcor(lazyload_semcor("data/corpora/semcor2.1/brown1/tagfiles/"));

In [7]:
function window(context, index::Int, window_size::Int=10)
    window_lower_bound = max(index - window_size÷2, 1)
    window_upper_bound = min(index + window_size÷2, length(context))
    view(context, [window_lower_bound:index-1 ; index+1:window_upper_bound])
end

function window(tagged_sense::TaggedSentence, index::Int, window_size::Int=10)
    context = lowercase.(strip_tags(tagged_sense))
    window(context, index, window_size)
end
   
function window(context, word::AbstractString, window_size::Int=10)
    context = lowercase.(context)
    occurances = find(context.==word)
    index =  length(occurances) > 0 ? occurances[ceil(Int, end/2)] : 0
    window(context, index, window_size)

end


window((drop(semcor,400) |> first |> last |> first)...)

window((synsets(db, db["fireplace", 'n']) |> first).gloss |> split, "fireplace")

10-element SubArray{String,1,Array{String,1},Tuple{Array{Int64,1}},false}:
 "fire"  
 "can"   
 "be"    
 "built;"
 "\"the" 
 "was"   
 "so"    
 "large" 
 "you"   
 "could" 

In [8]:

"Collect up the usages from a indexed tagged source"
function get_usages(usage_index::SemcorIndex, key)
    if haskey(usage_index, key)
        [window(lowercase.(strip_tags(sent)), index) for (sent, index) in usage_index[key]]
    else
        Vector{String}[]
    end
end

function get_usages(synset::Synset, lemma_word::AbstractString)
    gloss::Vector{SubString{String}} = lowercase.(punctuation_space_tokenize(synset.gloss))
    [window(gloss, lemma_word)]
end

function get_all_usages(wn::DB, semcor::SemcorIndex, lemma_word, pos)   
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    
    Dict{Synset,AbstractVector{AbstractVector}}((synset => #get_usages(synset)
        [get_usages(semcor, sensekey(db, synset, lemma)); get_usages(synset, lemma_word)]
                    for synset in target_synsets))  
    
end

get_all_usages (generic function with 1 method)

In [9]:
normal_probs(logprobs::Vector) = normal_probs(copy(logprobs))
function normal_probs!{F<:AbstractFloat}(logprobs::Vector{F})
    ret = logprobs
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end

function general_wsd(ee, context, wvs, priors=ones(length(wvs)))
    lps = Vector{Float64}(length(wvs))
    for (ii,(prior, wv)) in enumerate(zip(priors, wvs))
        lps[ii] = Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=true)
        lps[ii] += log(prior)
    end
    
    normal_probs!(lps)
end


general_wsd (generic function with 2 methods)

In [10]:
function synthesize_embedding1(ee,context::AbstractVector, 
                               word_or_phrase::AbstractString,
                               fallback_word_or_phrase=""::AbstractString)
    
    words = split(word_or_phrase, " ")
    wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
        fallbacks = split(fallback_word_or_phrase, " ")
        wvs = vcat((all_word_sense_vectors(ee,w) for w in fallbacks)...)
        if length(wvs) == 0
            throw(KeyError(" $words, nor $fallbacks have embeddings"))
        end
    end

    probs = general_wsd(ee, context, wvs)
    sum(wvs.*probs)
end
    

#function synthesize_embedding(am::AdaGramModel,context::AbstractVector, word::AbstractString)
#    known_context = filter(c->haskey(am.dict.word2id, c), context)
#    sum(all_word_sense_vectors(am, word).*disambiguate(am.vm, am.dict, word, known_context, false))
#end

synthesize_embedding1 (generic function with 2 methods)

###Note the disambig prob fumctions :
 - Query.logprob_of_context(am, context, wv; , normalise_over_length=false)
- Adagram.disambiguate(am.vm, am.dict, word, known_context, false))
Are identical. That is with normalise_over_length false for Logprob-of-context, 
and use_prior false for Adagram.disambiguate 

In [11]:
all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])

function _lexically_informed_embeddings(wn::DB,ee, word,lemma_word, pos)
    indexed_corpus = semcor #TODO: Pass this as a parameter, not as a gloel
    target_synset_examples = get_all_usages(wn, indexed_corpus, lemma_word, pos)
    target_synsets = collect(keys(target_synset_examples))
    lem = db[lemma_word, pos]
        
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,(synset, examples)) in enumerate(target_synset_examples)
        context::Vector{SubString{String}} = vcat(examples...)
        embeddings[ii] =synthesize_embedding1(ee,context, word, lemma_word)
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sensecount(db, ss, lem ) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lem
end

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
        
_li_embeddings = Dict{Tuple{DB, Any, String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Synset}, Lemma}}()
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    get!(_li_embeddings, (wn, ee, word, lemma_word, pos)) do
        _lexically_informed_embeddings(wn, ee, word, lemma_word, pos)
    end
end

lexically_informed_embeddings (generic function with 2 methods)

In [ ]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lem = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        
        priors = [float(sensecount(db, ss, lem)) for ss in target_synsets] #TODO use thing
        priors .+= 0
        priors .+= sqrt(sum(priors))/length(priors)
        priors ./= sum(priors)
        probs_of_sense = general_wsd(ee, challenge.context, embeddings, priors)
               
     
        sense_index= indmax(probs_of_sense)        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        @show ex
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lem = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lem)
        
        sense_freqs =  Float32[sensecount(db, ss, lem) for ss in target_synsets]
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lem)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



In [ ]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    @showprogress for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    @show notattempted
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, s_am, db))

#println("only nouns  : \t\t", 
#@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), s_am, db)
#println("only verbs  : \t\t", 
#@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), s_am, db)
#println("only adjecti: \t\t", 
#@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), s_am, db)
#println("only adverbs: \t\t", 
#@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), s_am, db)

# "models/adagram/more_senses.adagram_model.jld
### with Adagram Disambig weighting (prior used)
### with SemCor data 
overall: 		(0.655408489274304,0.6328779197884531,0.6439461883408072)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7161410018552876,0.6967509025270758,0.706312900274474)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5106382978723404,0.4873096446700508,0.4987012987012987)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6619718309859155,0.649171270718232,0.6555090655509065)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7268041237113402,0.6778846153846154,0.7014925373134329)

### with Adagram Disambig weighting (prior used)
### with just glosses 

overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
- nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
- vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
- aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
- rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)



# "semhuff_more_senses.adagram_model.jld

### with Adagram Disambig weighting (prior used)
### with just glosses 
 - overall: 		(0.6704701049748973,0.6474217717055972,0.658744394618834)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.7133580705009277,0.694043321299639,0.7035681610247027)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.5780141843971631,0.5516074450084603,0.5645021645021645)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6535211267605634,0.6408839779005525,0.6471408647140865)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.7319587628865979,0.6826923076923077,0.7064676616915424)
 
 ### with Adagram Disambig weighting (prior used)
 ### With semcore
 - overall: 		(0.6134185303514377,0.5923314235345968,0.6026905829596412)
 - nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),s_am,db) = (0.6252319109461967,0.6083032490974729,0.6166514181152789)
 - vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),s_am,db) = (0.5638297872340425,0.5380710659898477,0.5506493506493505)
 - aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),s_am,db) = (0.6169014084507042,0.6049723756906077,0.610878661087866)
 - rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),s_am,db) = (0.6855670103092784,0.6394230769230769,0.6616915422885572)
 
 
 # Most Frequent Sense
 
  -  overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
  - only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
  - only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
  - only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
  - only adverbs: 		(0.875,0.875,0.875)


In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

#println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
#println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
#println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
#println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


In [ ]:
#Zero shot WSI for 1 shot WSD
using Training

In [ ]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=50)
rr.corpus_size = ee.corpus_size
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr);

In [ ]:
rr.embedding["us"] |> length

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db))

In [ ]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db))

importfrom (generic function with 1 method)

funcdec = :(sensekey(##324::WordNet.DB,##325::WordNet.Synset,##326::WordNet.Lemma) = (WordNet).(sensekey)(##324,##325,##326))


In [74]:
mapping_corpus = load_semcor("data/corpora/semcor2.1/brown1/tagfiles/");

In [252]:
function agirresAlignment(ee, wn::DB, mapping_corpus, hard=true)
    maps = Dict{Tuple{String, Char}, Matrix{Float32}}()
    freqs = Dict{Tuple{String, Char}, Vector{Int}}()
    
    function proc_word(word::TaggedWord, sentence::TaggedSentence)
    end
    
    function proc_word(taggedword::SenseAnnotatedWord, sentence::TaggedSentence)
        local wn_sensekeys
        pos = pennPOStoWordNetPOS(taggedword.pos)
        

        
        try 
            wn_sensekeys = sensekeys(wn, wn[pos, taggedword.lemma])
        catch exception
            exception |> typeof <: KeyError || rethrow()
            warn("Could not find wordnet lemma for $taggedword")
            return
        end
        
        target_wnsn = findfirst(wn_sensekeys .== sensekey(taggedword))
        if target_wnsn==0
            warn("$(sensekey(taggedword)) not in  $(wn_sensekeys).\n\nIssue is with $taggedword")
            return
        end

        wvs = all_word_sense_vectors(ee,taggedword.word)
        if length(wvs) == 0
            warn("No embedding for $(taggedword.word); skipping")
            return
        end
        
        ########
        
        map = get!(maps, (taggedword.lemma, pos)) do
            zeros(length(wvs), length(wn_sensekeys))
        end
        
        freq = get!(freqs, (taggedword.lemma, pos)) do 
            zeros(Int, length(wn_sensekeys))
        end
        
        @assert(length(freq) == size(map,2) == length(wn_sensekeys),
        """
        $(size(map)) $(length(freq)) $(length(wn_sensekeys))
        $taggedword
        """ )
        @assert(size(map,1) == length(wvs))
        
        
        context = lowercase.(strip_tags(sentence))
        
        wv_probs = general_wsd(ee,context, wvs)
        @assert !any(isnan.(wv_probs))
        freq[target_wnsn] += 1
        if hard
            map[indmax(wv_probs),target_wnsn] += 1
        else
            map[:,target_wnsn] += wv_probs
        end
        
    end
    
    for sentence in mapping_corpus
        for word in sentence
            proc_word(word, sentence)
        end
    end
 
    if !(hard)
        for ((word,pos), freq) in freqs
            map = maps[(word,pos)]
            map ./= freq'
            map[isnan.(map)] = 0.0 #NaNs are just frequency 0 items
            @assert(all((isapprox(x,1.0; atol=10.0^-5)  for x in sum(map, 1))))
        end
    end
    
    maps, freqs
end

agirresAlignment (generic function with 2 methods)

In [ ]:
maps, freqs  = agirresAlignment(am, db, mapping_corpus, false);

In [221]:
m = maps["city",'n']
f = freqs["city",'n']
@show m
@show f

m = Float32[0.855639 0.177298 0.0171722; 0.909973 0.175451 0.0122244; 0.47952 0.0315287 0.00442265; 1.05545 0.265723 0.0104815; 0.884937 0.139869 0.0130272; 1.14186 0.248063 0.0270401; 1.04605 0.176553 0.0137959; 1.21186 0.247378 0.0108512; 1.13308 0.401118 0.0163113; 1.12129 0.216413 0.0197542; 0.999944 0.241073 0.0147291; 0.510784 0.020609 0.00859674; 1.1716 0.265569 0.021097; 1.13275 0.19544 0.0175935; 1.03405 0.208593 0.0139159; 0.849475 0.12992 0.0123045; 1.08369 0.168431 0.0107693; 3.08044 0.665897 0.0573607; 3.10473 0.668272 0.0582956; 3.10973 0.669066 0.058339; 3.10907 0.668374 0.0581404; 3.10823 0.668336 0.0584079; 3.10801 0.669025 0.0580926; 3.11235 0.669242 0.0580193; 3.11006 0.668676 0.0581764; 3.10551 0.669185 0.0581798; 3.10266 0.667512 0.058163; 3.10847 0.668792 0.0581324; 3.10971 0.669737 0.0581876; 3.10907 0.668856 0.0584186]
f = [57,12,1]


3-element Array{Int64,1}:
 57
 12
  1

In [249]:
isapprox(1.0,2.0; atol=10.0)

true

false

In [199]:
a = [1.0 2 3 0 ; 11 22 33 0 ; 111 222 333 0; 1111 2222 3333 0; 11111 22222 33333 5]
b = [5, -5, 1, 0]

@show a./b'

a./=b'
a[isnan(a)]=0.0
a

a ./ b' = [0.2 -0.4 3.0 NaN; 2.2 -4.4 33.0 NaN; 22.2 -44.4 333.0 NaN; 222.2 -444.4 3333.0 NaN; 2222.2 -4444.4 33333.0 Inf]


5×4 Array{Float64,2}:
    0.2     -0.4      3.0    0.0
    2.2     -4.4     33.0    0.0
   22.2    -44.4    333.0    0.0
  222.2   -444.4   3333.0    0.0
 2222.2  -4444.4  33333.0  Inf  

In [188]:
n = copy(m)
@show n
@show f
n./=f'
n[isnan.(n)]=0.0
n

n = Float32[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 57.0 12.0 1.0]
f = [57,12,1]


30×3 Array{Float32,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  1.0  1.0

In [150]:
isnan.([NaN 1; Inf 4])

2×2 Array{Bool,2}:
  true  false
 false  false

In [100]:
ww = CorpusLoaders.Semcor.SenseAnnotatedWord{SubString{String}}("NN","funds",1,"1:21:00::","funds")
sensekey(ww)

"funds%1:21:00::"

In [103]:
@show ww.pos 
ww.lemma

ww.pos = "NN"


"funds"

In [ ]:
methods(sensekey)